# 1. Basic Settings

In [1]:
import gensim
import os
import logging

# Log settings
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


c:\nlp\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
# Note that this imports ALL FILES in dir
class MySentences(object):

    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield gensim.utils.simple_preprocess(line)

# a memory - friendly iterator
sentences = MySentences('D:\\coca and GloWbE\\coca\\temp\\text2doc')



# 2. (skip) Preprocessing - Dividing text in file into Docs

Each file has hundreds of documents, with number tags leading the start of each article 
i.e) ##300000, ##300001, #300002 . . .

In [6]:
for i in range(1990, 2013): # newspaper files from 1990 to 2013
    file = open("D:\\COCA & GloWbE\\COCA\\texts\\text_newspaper_lsp\\w_news_%d.txt" % i)
    newtext = file.read()
    print(newtext[1:100], "\n")
    file.close()

##3000001 <p> He is trying to make the best of it , but the days have seemed to pass like months fo 

##3000061 <p> N.B.A. Hawks Streak Past Lakers Dominique Wilkins scored 34 points to spark Atlanta o 

##3000125 <p> Since leaping in a single bound to the pinnacle of the golf world , John Daly has fou 

##3000191 <p> Joe Caldwell lives in 1993 , but he is stuck in 1974 , when the former Atlanta Hawk ' 

##3000255 <p> Eleven years ago , Tom Moe Sr. and son Tommy made a father-and-son connection , layin 

##3000323 <p> As a unique custody battle raged round him late last week , Ch . Tobe 's Return of th 

##3000393 <p> As concern grew in the boxing community that Tommy Morrison had tested positive for H 

##3000462 <p> Ending one of pro football 's more confounding legal scrimmages , National Football L 

##3000526 <p> Casey Martin won the right today to use a cart in professional golf tournaments , bec 

##3000589 <p> Some teams improve by adding players . Others make themselves better

# Segement each yearly file to seperate documents

In [3]:
import re

doc_index = 1

for i in range(1990, 2013):
    file = open("D:\\coca and GloWbE\\coca\\texts\\text_newspaper_lsp\\w_news_%d.txt" % i)
    newtext = file.read()
    file.close()
    
    p = re.split('\n##\d+', newtext)
    p = p[1:] # due to list.split() the 1st item is blank
    print("File w_news_%d" % i, "-", "Number of files : ", len(p))

    # Using RegEx & Storing each article as a file
    
    for f in p:
        written = open(("D:\\coca and GloWbE\\coca\\temp\\text2doc\\file_%d.txt" % doc_index), 'w')
        written.write(f)

        doc_index+= 1
        
print("Total number of files:", doc_index)

File w_news_1990 - Number of files :  2830
File w_news_1991 - Number of files :  2869
File w_news_1992 - Number of files :  2478
File w_news_1993 - Number of files :  2551
File w_news_1994 - Number of files :  2380
File w_news_1995 - Number of files :  2451
File w_news_1996 - Number of files :  2325
File w_news_1997 - Number of files :  2409
File w_news_1998 - Number of files :  2376
File w_news_1999 - Number of files :  2445
File w_news_2000 - Number of files :  2193
File w_news_2001 - Number of files :  2117
File w_news_2002 - Number of files :  2290
File w_news_2003 - Number of files :  2251
File w_news_2004 - Number of files :  2251
File w_news_2005 - Number of files :  2410
File w_news_2006 - Number of files :  2369
File w_news_2007 - Number of files :  2850
File w_news_2008 - Number of files :  2868
File w_news_2009 - Number of files :  2871
File w_news_2010 - Number of files :  3026
File w_news_2011 - Number of files :  2868
File w_news_2012 - Number of files :  1559
Total numbe

# 3. Training - Import file list

In [12]:
from os import listdir
from os.path import isfile, join
docLabels = []
docLabels = [f for f in listdir("D:\\coca and GloWbE\\coca\\temp\\text2doc") if f.endswith('.txt')]


# Doc labels (same as file name)

In [13]:
data = []

for doc in docLabels[0:500]:
    data.append( str((open("D:\\coca and GloWbE\\coca\\temp\\text2doc\\" + doc).readlines(), 'r')))

# Labeled Sentence interator

In [16]:
# import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence
preprocess = gensim.utils.simple_preprocess


class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=preprocess(doc),tags=[self.labels_list[idx]])

# Training

In [17]:
sentences = LabeledLineSentence(data, docLabels)

In [ ]:
# This is too slow to run
model = gensim.models.Doc2Vec(size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(sentences)
for epoch in range(10):
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
#     model.train(it, total_examples=model.corpus_count, epochs=model.iter)

2017-09-16 18:01:13,564 : INFO : collecting all words and their counts
2017-09-16 18:01:13,567 : WARNING : Each 'words' should be a list of words (usually unicode strings).First 'words' here is instead plain <class 'str'>.
2017-09-16 18:01:13,569 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-09-16 18:01:14,097 : INFO : collected 86 word types and 500 unique tags from a corpus of 500 examples and 4523335 words
2017-09-16 18:01:14,099 : INFO : Loading a fresh vocabulary
2017-09-16 18:01:14,100 : INFO : min_count=5 retains 86 unique words (100% of original 86, drops 0)
2017-09-16 18:01:14,102 : INFO : min_count=5 leaves 4523335 word corpus (100% of original 4523335, drops 0)
2017-09-16 18:01:14,104 : INFO : deleting the raw counts dictionary of 86 items
2017-09-16 18:01:14,108 : INFO : sample=0.001 downsamples 36 most-common words
2017-09-16 18:01:14,111 : INFO : downsampling leaves estimated 1062345 word corpus (23.5% of prior 4523335)
2017-09-16 18

In [10]:
# This is too slow to run
model = gensim.models.Doc2Vec(size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
for epoch in range(10):
    model.train(it, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it, total_examples=model.corpus_count, epochs=model.iter)

2017-08-26 15:18:09,876 : INFO : collecting all words and their counts
2017-08-26 15:18:09,880 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-08-26 15:18:36,162 : INFO : PROGRESS: at example #10000, processed 14850100 words (565078/s), 141875 word types, 10000 tags
2017-08-26 15:19:03,094 : INFO : PROGRESS: at example #20000, processed 30072334 words (565233/s), 197396 word types, 20000 tags
2017-08-26 15:19:31,993 : INFO : PROGRESS: at example #30000, processed 45791379 words (543948/s), 249775 word types, 30000 tags
2017-08-26 15:19:55,645 : INFO : PROGRESS: at example #40000, processed 59188237 words (566439/s), 292614 word types, 40000 tags
2017-08-26 15:20:18,096 : INFO : PROGRESS: at example #50000, processed 71461876 words (546719/s), 332489 word types, 50000 tags
2017-08-26 15:20:34,963 : INFO : collected 354210 word types and 57037 unique tags from a corpus of 57037 examples and 81019332 words
2017-08-26 15:20:34,964 : INFO : Loading a fre

2017-08-26 15:21:43,591 : INFO : PROGRESS: at 4.88% examples, 260077 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:44,602 : INFO : PROGRESS: at 4.96% examples, 260417 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:45,644 : INFO : PROGRESS: at 5.03% examples, 260448 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:46,653 : INFO : PROGRESS: at 5.11% examples, 260396 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:47,668 : INFO : PROGRESS: at 5.20% examples, 260613 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:48,701 : INFO : PROGRESS: at 5.29% examples, 260835 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:49,702 : INFO : PROGRESS: at 5.37% examples, 260923 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:50,729 : INFO : PROGRESS: at 5.45% examples, 261019 words/s, in_qsize 21, out_qsize 0
2017-08-26 15:21:51,737 : INFO : PROGRESS: at 5.53% examples, 261034 words/s, in_qsize 22, out_qsize 0
2017-08-26 15:21:52,772 : INFO : PROGRESS: at 5.59% examples, 260276 word

KeyboardInterrupt: 

In [ ]:
model = gensim.models.Doc2Vec(alpha=0.025, min_alpha=0.025)  # use fixed learning rate
model.build_vocab(sentences)
for epoch in range(10):
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

In [44]:
model.wv.vocab

{'he': <gensim.models.keyedvectors.Vocab at 0x16773ab5080>,
 'is': <gensim.models.keyedvectors.Vocab at 0x16773c34e10>,
 'trying': <gensim.models.keyedvectors.Vocab at 0x16773c34208>,
 'to': <gensim.models.keyedvectors.Vocab at 0x16773c342b0>,
 'make': <gensim.models.keyedvectors.Vocab at 0x16773c34630>,
 'the': <gensim.models.keyedvectors.Vocab at 0x16773c346d8>,
 'best': <gensim.models.keyedvectors.Vocab at 0x16773c344e0>,
 'of': <gensim.models.keyedvectors.Vocab at 0x16773c34160>,
 'it': <gensim.models.keyedvectors.Vocab at 0x16773c34e80>,
 'but': <gensim.models.keyedvectors.Vocab at 0x16773c34898>,
 'days': <gensim.models.keyedvectors.Vocab at 0x16773c347b8>,
 'have': <gensim.models.keyedvectors.Vocab at 0x16773c34d30>,
 'seemed': <gensim.models.keyedvectors.Vocab at 0x16773c349e8>,
 'pass': <gensim.models.keyedvectors.Vocab at 0x16773c34a58>,
 'like': <gensim.models.keyedvectors.Vocab at 0x16773c34c88>,
 'months': <gensim.models.keyedvectors.Vocab at 0x16773a4d0f0>,
 'for': <gensi

# 4. Save or Load model

In [23]:
model.save("C:\\NLP\\Pre-trained\\Doc2Vec(news1990~2012-preprocessed).bin")

2017-08-26 19:35:12,287 : INFO : saving Doc2Vec object under C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin, separately None
2017-08-26 19:35:12,289 : INFO : storing np array 'syn0' to C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.wv.syn0.npy
2017-08-26 19:35:12,452 : INFO : not storing attribute syn0norm
2017-08-26 19:35:12,453 : INFO : storing np array 'syn1neg' to C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.syn1neg.npy
2017-08-26 19:35:12,610 : INFO : not storing attribute cum_table
2017-08-26 19:35:13,805 : INFO : saved C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin


In [3]:
# model = gensim.models.Doc2Vec.load("C:\\NLP\\Pre-trained\\Doc2Vec.bin")
# model = gensim.models.Doc2Vec.load("C:\\NLP\\Pre-trained\\Doc2Vec(news1990-preprocessed).bin")
model = gensim.models.Doc2Vec.load("C:\\NLP\\Pre-trained\\Doc2Vec(news1990~2012-preprocessed).bin")

2017-08-27 15:19:28,339 : INFO : loading Doc2Vec object from C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin
2017-08-27 15:19:29,536 : INFO : loading wv recursively from C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.wv.* with mmap=None
2017-08-27 15:19:29,537 : INFO : loading syn0 from C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.wv.syn0.npy with mmap=None
2017-08-27 15:19:29,670 : INFO : setting ignored attribute syn0norm to None
2017-08-27 15:19:29,671 : INFO : loading docvecs recursively from C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.docvecs.* with mmap=None
2017-08-27 15:19:29,673 : INFO : loading syn1neg from C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin.syn1neg.npy with mmap=None
2017-08-27 15:19:29,845 : INFO : setting ignored attribute cum_table to None
2017-08-27 15:19:29,848 : INFO : loaded C:\NLP\Pre-trained\Doc2Vec(news1990~2012-preprocessed).bin


# 5. Look-up for similar documents

In [8]:
vector = model.infer_vector(""" <p> He is trying to make the best of it , but the days have seemed to pass like months for Lou Piniella . Some days , he visits relatives . Others , he spends the afternoon fishing . Mostly , he waits . It has been like this since the first week of February , when Piniella and his family arrived at their beach home in Reddington Shores , Fla. , and waited for baseball to begin . But there has been only stillness . <p> The owners ' lockout of spring camps , now in its second week , has kept Piniella from beginning his tenure as the new manager of the Cincinnati Reds . Until he does , he will still seem like a Yankee , even though all of that is officially behind him , once and for all . Putting 16 Years Behind <p> " It was different , " Piniella said today by phone , recalling the first time he put on the Reds ' uniform and looked at himself . " It seemed like I was engulfed in a sea of @ @ @ @ @ @ @ @ @ @ . It was a part of me . When I put on the red stockings and red shirt and red cap , I had to get used to it . It will take a while . " <p> When Piniella left New York to sign a three-year deal as the Reds ' manager , succeeding Pete Rose , he walked away from 16 years with the Yankees . He had been a player , coach , two-time manager , general manager and broadcaster . He is still the last person to manage the team for a full season and the last to manage it in consecutive years , winning 90 games in 1986 and 89 the next year . <p> But Piniella said he felt a greater need to separate himself from the Yankee machinery , from the impression that he was pulled by George Steinbrenner , the principal owner . No Longer ' George 's Guy ' <p> " I could have stayed with the Yankees , " he said . " But I was seen in baseball as being George 's guy , and I did n't want that @ @ @ @ @ @ @ @ @ @ and the things he 's accomplished , but I needed to go out on my own and get something done . If you 're not out there doing things , it 's easy to fade from the picture . I did n't want to be thought of as a recycled manager . " <p> It was Bob Quinn , the former Yankee general manager now in a similar post with the Reds , who took a major role in bringing Piniella to Cincinnati . Like Piniella , Quinn had spent a number of years with the Yankees and Steinbrenner before sensing the urge to break away . <p> " Lou probably felt that this was his chance to establish his own identity , " Quinn said , " without having to be Lou III or Lou IV . " <p> But Piniella will not be without some pressure in Cincinnati . Rose 's departure , the result of a ban resulting from his gambling activities , may cause problems for the new manager . Legends , even those with soiled pasts , are not succeeded easily . <p> " I really @ @ @ @ @ @ @ @ @ @ . " I admire him and the heights he 's attained , but I do n't know him . I went to Plant City a few weeks ago and we spoke for a few minutes , but that was it . " <p> Rose , who managed the Reds for almost five full seasons , will do daily commentary for a Cincinnati radio station during the season , keeping himself widely visible . Open for Input <p> Asked if he would consider seeking Rose 's advice , Piniella said : " He knows this ball club . He managed it . The more input , the better . " <p> Quinn said of any possible pressure : " I do n't see it bothering Lou at all . If Lou could handle the New York situation so capably , I do n't think anything here could be of concern to him . " <p> And a source close to the team said of Rose 's shadow : " The perception of pressure could be presented , but I do n't view it that way . Pete is not ready for canonization @ @ @ @ @ @ @ @ @ @ success will depend solely on the team 's rise or fall . Under Rose , the Reds finished second in the National League West four straight years . Then , as baseball 's investigation lowered a net around their manager last summer , they dropped to fifth . <p> " We feel we 've got a chance to win the division , " Piniella said . " They 've come close but fallen short . The reports I 've read said we 've got good talent . I 've watched the team on television , and we have some quality players like Eric Davis and Barry Larkin . I think the oldest player we 'll put on the field will be 29 . " Factors in the Fall <p> Rose 's situation may have contributed to the Reds ' fall last season , but so did injuries to such players as Larkin , Danny Jackson and Jose Rijo . Piniella said he avoided talk of Rose in his off-season discussions with several players . <p> " I did n't dwell on that , " he said , " but certainly it @ @ @ @ @ @ @ @ @ @ year 's Cincinnati club , the thing that contributed to their bad performance was so many injuries . It 's hard to compete . " <p> Piniella has attempted to offset his lack of knowledge about the National League by supporting himself with a coaching staff familiar with it . He kept Tony Perez from the previous regime , brought in Sam Perlozzo from the Mets to coach third base , hired Jackie Moore from the Montreal Expos as his bench coach and Stan Williams , the former Yankee aide , as his pitching coach . He also added Larry Rothschild , who had spent time as the Reds ' roving minor league pitching instructor . <p> Best of all , he expects no interference from the team 's owner , Marge Schott , who has been known to be as unpredictable and as demanding as Steinbrenner . <p> " I have n't had any troubles with Marge , " Piniella said . " She 's confident with me as the manager , and I plan to communicate with her and offer my assessments of what 's going on : good @ @ @ @ @ @ @ @ @ @ <p> But so far , Piniella has been forced to keep his distance from the team 's Plant City complex , which is less than two hours from his home on the Florida gulf . <p> " We were supposed to start on Feb. 18 , but I 've just been sitting around waiting , " he said . " I have n't been doing much of anything except seeing friends in the Tampa area and seeing my folks and in-laws . I 've done some fishing and spent a lot of lazy time . " <p> When it does begin , the Yankee portion of his life will be gone . <p> " I 'm always going to pick up a paper and see how they 're doing , " he said . " I 'll be rooting for them . They were a big part of me . There 's nothing more I 'd like to do than get our club to the post-season and see the Yankees do the same thing . " Still , he can not see himself coming back . " I came to Cincinnati @ @ @ @ @ @ @ @ @ @ can , " he said . " I do n't want to bounce around in baseball . This is a stable situation , and if I do a job I 'll be around for a long time . 
" """.split())
model.docvecs.most_similar([vector]) # similar docs & accuracy

[('file_1.txt', 0.8102234601974487),
 ('file_5732.txt', 0.6845009326934814),
 ('file_31743.txt', 0.6783349514007568),
 ('file_15564.txt', 0.6781762838363647),
 ('file_7983.txt', 0.6721680164337158),
 ('file_8181.txt', 0.6701405644416809),
 ('file_16290.txt', 0.6677618026733398),
 ('file_10375.txt', 0.6590215563774109),
 ('file_22104.txt', 0.6589066386222839),
 ('file_22689.txt', 0.6566866636276245)]

In [10]:
similar_docs = model.docvecs.most_similar([vector])
file_names = [i[0] for i in similar_docs] # slice the first elements
results = []
        
for name in file_names:
    text = open("D:\\coca and GloWbE\\coca\\temp\\text2doc\\" + name, 'r')
    results.append(text.read())
    text.close()
        
i = 0

for text in results:
    print("Document", i+1, ":", file_names[i], "\t accuracy:", similar_docs[i][1], "\n", (text[0:500]), "\n" ) 
    i += 1

Document 1 : file_1.txt 	 accuracy: 0.8102234601974487 
  <p> He is trying to make the best of it , but the days have seemed to pass like months for Lou Piniella . Some days , he visits relatives . Others , he spends the afternoon fishing . Mostly , he waits . It has been like this since the first week of February , when Piniella and his family arrived at their beach home in Reddington Shores , Fla. , and waited for baseball to begin . But there has been only stillness . <p> The owners ' lockout of spring camps , now in its second week , has kept Pini 

Document 2 : file_5732.txt 	 accuracy: 0.6845009326934814 
  <p> It seemed like a good time for a celebration . <p> Buck Rodgers was turning 54 years old , he was feeling better every day and he was with his family , enjoying a bright summer day near the pool . <p> But last Sunday , Rodgers -- the California Angels ' manager who is still mending from injuries he suffered in a team bus accident May 21 on the New Jersey Turnpike -- sat in